In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# 태안해안국립공원 영역을 추출하는 함수 정의
def get_boundary():
    roi = ee.FeatureCollection("WCMC/WDPA/current/polygons") \
        .filter(ee.Filter.eq("NAME", "Taeanhaean")) \
        .geometry()
    return roi

# 태안해안국립공원 경계 추출
region = get_boundary()

# 중앙점 계산
centroid = region.centroid()

# 중앙점의 위도와 경도 값 얻기
lon_lat = centroid.getInfo()['coordinates']
longitude = lon_lat[0]
latitude = lon_lat[1]

# 위도와 경도 값을 소수점 넷째 자리까지 출력
print("중앙점의 위도: {:.4f}".format(latitude))
print("중앙점의 경도: {:.4f}".format(longitude))

중앙점의 위도: 36.6125
중앙점의 경도: 126.2256


In [4]:
point = ee.Geometry.Point([126.2256, 36.6125])

image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
    .filterBounds(point)
    .filterDate('2022-01-01', '2022-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 0.4, 'bands': ['B5', 'B4', 'B3']}

# Landsat-8 데이터 추가
Map.centerObject(point, 9)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [5]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2022-10-27'

In [6]:
image.get('CLOUD_COVER').getInfo()

0.02

In [7]:
# ESA WorldCover 10m v200
esa_worldcover = ee.ImageCollection('ESA/WorldCover/v200').first()
esa_worldcover_kr = esa_worldcover.clip(image.geometry())

Map.addLayer(esa_worldcover_kr, {}, 'ESA WorldCover')
Map

Map(center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [8]:
# 학습 데이터 생성
points = esa_worldcover_kr.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,
    }
)

Map.addLayer(points, {}, 'training', False)

In [9]:
print(points.size().getInfo())

4103


In [10]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [125.95303141182198, 36.149676107086066]}, 'id': '0', 'properties': {'Map': 80}}


In [11]:
# 예측에 사용하는 밴드
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# 토지피복 레이블 저장 테이블의 속성
label = 'Map'

# 이미지 위에 점들을 오버레이하여 훈련 데이터 얻기
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# 기본 매개변수를 사용하여 CART 분류기 훈련
trained = ee.Classifier.smileCart().train(training, label, bands)

In [12]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.1613909751176834, 'B2': 0.13543030619621277, 'B3': 0.08888012915849686, 'B4': 0.05841150879859924, 'B5': 0.03379363194108009, 'B6': 0.011541628278791904, 'B7': 0.006362282671034336, 'Map': 80}}


In [13]:
# 훈련에 사용된 동일한 밴드로 이미지 분류
result = image.select(bands).classify(trained)

# 랜덤한 색상으로 클러스터 표시
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [14]:
class_values = esa_worldcover_kr.get('Map_class_values').getInfo()
class_values

[10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]

In [15]:
class_palette = esa_worldcover_kr.get('Map_class_palette').getInfo()
class_palette

['006400',
 'ffbb22',
 'ffff4c',
 'f096ff',
 'fa0000',
 'b4b4b4',
 'f0f0f0',
 '0064c8',
 '0096a0',
 '00cf75',
 'fae6a0']

In [16]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [17]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [18]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [19]:
Map.add_legend(builtin_legend='ESA_WorldCover')
Map

Map(center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [20]:
import os

out_dir = 'D:/GEODATA'
out_file = os.path.join(out_dir, 'landcover.tif')

geemap.ee_export_image(landcover, filename=out_file, scale=100)

"""
geemap.ee_export_image_to_drive(
    landcover, description='landcover', folder='export', scale=30
)
"""

Generating URL ...
Please wait ...
Data downloaded to D:\GEODATA\landcover.tif


"\ngeemap.ee_export_image_to_drive(\n    landcover, description='landcover', folder='export', scale=30\n)\n"